In [1]:
from experiment import Experiment
from utils.behavior_data import BehaviorData
from visuals import Plotter
import torch
import numpy as np
from utils.state_data import StateData
import matplotlib.pyplot as plt

In [ ]:
# model, learning_rate = "AdaptableLSTM", .025
model, learning_rate = "BasicNN", .0054
# model, learning_rate = "LogisticRegressor", .003
epochs = 1000
seed = 1
include_state = True
estate = True
fullq = True
respond_perc = .50
fullseq = True
insertpreds = True
noise = 0
smooth = .1

    
np.random.seed(seed)
torch.manual_seed(seed)


e = Experiment(
    numValFolds = 5,
    epochsToUpdateLabelMods = 10,
    data_kw={"minw": 2,
            "maxw": 31,
            "include_state": include_state,
            "include_pid": False,
            "expanded_states": estate,
            "top_respond_perc": respond_perc,
             "full_questionnaire": fullq,
             "full_sequence": fullseq,
             "insert_predictions": insertpreds,
             "one_hot_response_features": False,
             "response_feature_noise": noise
            },
    model=model,
    model_kw={
        "lossfn": "MSELoss",
        # "lossfn": "NDCG",
        # "lossfn": "CrossEntropyLoss",
        "hidden_size": 15,
        "lr_step_mult": .9, 
        "lr_step_epochs": 100,
        "opt_kw": {
            "lr": learning_rate,
        },
        "labelSmoothPerc": smooth,
        "gaussianNoiseStd": noise
        
    },
    train_kw={
        "epochs": epochs,
        "n_subj": 500,
        "rec_every": 5,
    })


print(len(e.bd.test))
print(len(e.bd.train))

report = e.run()






268 537
paction_sids
pmsg_ids
qids
response
(6456, 928) (6456, 8) 928
54
214
0	 train loss: 0.1635 train acc: 45.717% test acc: 43.603%
5	 train loss: 0.1488 train acc: 49.680% test acc: 48.250%
10	 train loss: 0.1435 train acc: 51.854% test acc: 50.488%
15	 train loss: 0.1437 train acc: 52.189% test acc: 50.258%
20	 train loss: 0.1410 train acc: 52.045% test acc: 50.660%
25	 train loss: 0.1389 train acc: 52.780% test acc: 50.660%
30	 train loss: 0.1369 train acc: 53.116% test acc: 51.061%
35	 train loss: 0.1352 train acc: 53.388% test acc: 51.291%
40	 train loss: 0.1337 train acc: 54.634% test acc: 50.832%
45	 train loss: 0.1322 train acc: 55.513% test acc: 50.717%
50	 train loss: 0.1306 train acc: 56.488% test acc: 50.373%
55	 train loss: 0.1291 train acc: 57.143% test acc: 50.373%
60	 train loss: 0.1276 train acc: 57.686% test acc: 50.258%
65	 train loss: 0.1266 train acc: 58.357% test acc: 50.086%
70	 train loss: 0.1249 train acc: 59.092% test acc: 50.545%
75	 train loss: 0.1236 tr

In [ ]:

print (np.mean(report['train_metrics'], axis=0))
labels = report["metric_labels"]
print(report['train_metrics'][-1, labels.index("Acc")])
print(report['test_metrics'][-1, labels.index("Acc")])

splot = plt.plot(report["rec_epochs"], report["train_metrics"][:, labels.index("Acc")], label="Train Acc.")
splot = plt.plot(report["rec_epochs"], report["test_metrics"][:, labels.index("Acc")], label="Test Acc.")
splot = plt.plot(report["rec_epochs"], report["train_metrics"][:, labels.index("MSE")], label="Train MSE")
splot = plt.plot(report["rec_epochs"], report["test_metrics"][:, labels.index("MSE")], label="Test MSE")
plt.title("Train/Test Performance Over Training")
plt.legend()
plt.ylabel("Metric")
plt.xlabel("Training Epoch")
plt.savefig("simpleNotebookAccPlot.png")

plt.clf()

In [ ]:
print(e.bd.features.shape, e.bd.featureList.shape)